In [1]:
inputRDD = sc.textFile("/FileStore/tables/dataSet10.txt")
inputRDD.first()

In [2]:
filteredRDD = inputRDD.filter(lambda line: len(line.strip())>0)
filteredRDD.take(3)

In [3]:
import re

def removestuff(verses):
  return re.sub(r'[,.:;!()?]+', '', verses.lower().strip())

kvPair = filteredRDD.map(lambda line: line.strip().split(' ', 1))
kvPair = kvPair.flatMap(lambda line: [line[0] + '_' + removestuff(line[1])])
kvPair = kvPair.map(lambda line: line.split('_', 1)).map(lambda line: [line[0], line[1].strip().split(' ')])
kvPair.take(3)

In [4]:
def pairs(line):
  templist = []
  for v in line[1]:
    templist.append(v + '_' + line[0])
  return templist

pairslist = kvPair.flatMap(lambda line: pairs(line)).map(lambda line: line.strip().split('_'))
pairslist.take(3)

In [5]:
inverted = pairslist.map(lambda i: (i[0], i[1])).distinct()
inverted.take(3)

In [6]:
output = inverted.groupBy(lambda x: x[0])
output = output.sortByKey(ascending=True)
output = output.map(lambda x: (x[0], [i[1] for i in list(x[1])]))
output = output.filter(lambda line: len(line[0].strip())>0)
output.take(3)

In [7]:
def sortVerseIdList(verseIdList):
  newlist = []
  for verseId in verseIdList:
    book, chapter, verse = verseId.split(":")
    newlist += [(book, int(chapter), int(verse))]
  newlist.sort()
  verseIdList = []
  for book, chapter, verse in newlist:
    verseId = book + ':' + str(chapter) + ':' + str(verse)
    verseIdList += [ verseId ]
  return verseIdList

output = output.map(lambda x: (x[0], sortVerseIdList(x[1])))
output.collect()



In [8]:
output.coalesce(1).saveAsTextFile("/mnt/S3/output/sparkhw2fullagain")

In [9]:
#extracredit
output = output.sortBy(lambda (x, y): (-len(y), x))
output.collect()

In [10]:
output.coalesce(1).saveAsTextFile("/mnt/S3/output/sparkhw2EC")